# A origem dos Jogadores

*Importando as Bibliotecas*

In [1]:
import pandas as pd
import numpy as np
from scipy import stats
import seaborn as sns
import matplotlib.pyplot as plt

### 1.Limpeza e Organização do Dataset

In [2]:
data = pd.read_csv('archive/male_players.csv',low_memory=False)

#### 1.1.Filtrando os dados relevantes para análises estatísticas

In [3]:
# Verificando se existe valores nulos
display(data.isnull().sum())
display(data.info())

player_id       0
player_url      0
fifa_version    0
fifa_update     0
update_as_of    0
               ..
lcb             0
cb              0
rcb             0
rb              0
gk              0
Length: 109, dtype: int64

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 180021 entries, 0 to 180020
Columns: 109 entries, player_id to gk
dtypes: float64(20), int64(43), object(46)
memory usage: 149.7+ MB


None

Como não existe valores nulos, não se faz necessária a limpeza.

In [5]:
#Removendo dados que não são relevantes para nossa análise

columns_irrelevant = ["fifa_update","value_eur","wage_eur","age","height_cm","weight_kg","player_url",
                      "update_as_of", "short_name", "long_name", "player_positions",'potential',
                      "dob", "club_team_id", "club_name", "league_id", "league_name", "league_level", "club_position", "club_jersey_number",
                      "club_loaned_from", "club_joined_date", 
                      "club_contract_valid_until_year", "nationality_id", "nationality_name", "nation_team_id", "nation_position", "nation_jersey_number",
                      "preferred_foot", "weak_foot", "skill_moves", "international_reputation", "work_rate", "body_type", "real_face", 
                      "release_clause_eur", "player_tags", "player_traits","player_id"]

players_attributes = data.drop(columns=columns_irrelevant)
players_attributes

,fifa_version,overall,pace,shooting,passing,dribbling,defending,physic,attacking_crossing,attacking_finishing,...,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk
0,24.0,91,97.0,90.0,80.0,92.0,36.0,78.0,78,94,...,63+3,63+3,63+3,68+3,63+3,54+3,54+3,54+3,63+3,18+3
1,24.0,91,89.0,93.0,66.0,80.0,45.0,88.0,47,96,...,63+3,63+3,63+3,62+3,60+3,62+3,62+3,62+3,60+3,19+3
2,24.0,91,72.0,88.0,94.0,87.0,65.0,78.0,95,85,...,80+3,80+3,80+3,79+3,75+3,70+3,70+3,70+3,75+3,21+3
3,24.0,90,80.0,87.0,90.0,94.0,33.0,64.0,83,89,...,63+3,63+3,63+3,64+3,59+3,49+3,49+3,49+3,59+3,19+3
4,24.0,90,79.0,88.0,83.0,87.0,39.0,78.0,75,91,...,64+3,64+3,64+3,64+3,60+3,55+3,55+3,55+3,60+3,18+3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
180016,15.0,41,66.0,25.0,29.0,30.0,39.0,53.0,25,25,...,38,38,38,37,40,41,41,41,40,10
180017,15.0,41,64.0,41.0,27.0,35.0,27.0,41.0,22,47,...,30,30,30,31,32,31,31,31,32,14
180018,15.0,40,58.0,27.0,35.0,31.0,52.0,68.0,49,24,...,46,46,46,48,51-1,53-3,53-3,53-3,51-1,12
180019,15.0,40,72.0,27.0,27.0,30.0,35.0,61.0,23,25,...,36,36,36,37,39,40,40,40,39,13


#### 1.2.Divindo o Dataset por categoria

Dividimos o Dataset por setores, escolhendo as habilidades que são mais relevantes para cada área.

##### 1.2.1.Categoria Defensiva

A categoria defensiva vai englobar o goleiro, os zagueiros, os laterais e os alas defensivos. 
Engloba os jogares responsáveis por defender e recuperar a posse da bola.

Para os laterais consideramos a velocidade como atributo mais importante. Para os zagueiros escolhemos a noção defensiva, pois sua responsabilidade defensiva é maior que a dos laterais.

No goleiro consideramos os reflexos como o maior impactante no desempenho.

In [7]:

defense_players = data.loc[data["player_positions"].isin(["GK", "CB", "LB", "RB","LWB","RWB"])]
defense_players = defense_players[["fifa_version", "player_positions", "short_name",
                                   "movement_sprint_speed","defending_marking_awareness","goalkeeping_reflexes" ]]

defense_players

,fifa_version,player_positions,short_name,movement_sprint_speed,defending_marking_awareness,goalkeeping_reflexes
6,24.0,GK,T. Courtois,52,20,93
9,24.0,GK,Alisson,49,15,89
11,24.0,CB,Rúben Dias,69,91,12
13,24.0,GK,M. ter Stegen,50,25,91
15,24.0,CB,V. van Dijk,87,90,11
...,...,...,...,...,...,...
180011,15.0,GK,B. Richardson,25,25,31
180013,15.0,GK,L. Pilling,41,25,45
180015,15.0,GK,S. Ramsbottom,40,25,44
180016,15.0,CB,E. Redman,66,38,6


##### 1.2.2.Categoria de Meio-Campo

 A categoria Meio-Campo vai englobar os meia-centrais, os volantes, os meia-abertas e os meia-atacantes.
 Engloba os jogares responsáveis pelo jogo rodar.

Para os volantes, meia-centrais e meia-atacantes consideramos o passe curto o atributo mais importante, já que são os maiores responsáveis por essa função. 

Para os meia-abertos escolhemos o drible pois, assim como os pontas, eles fazem mais jogadas individuais.

In [9]:
midfield_players = data.loc[data["player_positions"].isin(["CDM", "LM", "RM", "CM","CAM"])]
midfield_players = midfield_players[["fifa_version", "player_positions", "short_name",
                                   "attacking_short_passing","dribbling"]]

midfield_players

,fifa_version,player_positions,short_name,attacking_short_passing,dribbling
14,24.0,CDM,Casemiro,84,73.0
33,24.0,CM,L. Modrić,91,87.0
37,24.0,CM,Pedri,88,88.0
45,24.0,CM,N. Barella,86,85.0
60,24.0,CM,S. Milinković-Savić,86,82.0
...,...,...,...,...,...
179995,15.0,CM,A. Murdoch,40,57.0
179999,15.0,CM,G. Casey,50,46.0
180003,15.0,CM,K. Nasser,52,48.0
180006,15.0,CM,T. Smith,54,44.0


##### 1.2.3.Categoria Ataque

A categoria Ataque engloba os jogadores encarregados de finalizar as jogadas. Sendo eles os pontas, centroavantes e segundo-atacantes.

Para os pontas, escolhemos o drible pois, assim como os meia-abertos, eles fazem mais jogadas individuais.

Para os centroavantes e segundo-atacantes escolhemos a finalização, porque têm como papel principal finalizar as jogadas, ou seja, fazer o gol.

In [10]:
attack_players = data.loc[data["player_positions"].isin(["LW", "ST", "RW", "CF"])]
attack_players = attack_players[["fifa_version", "player_positions", "short_name",
                                   "attacking_finishing","dribbling"]]

attack_players

,fifa_version,player_positions,short_name,attacking_finishing,dribbling
1,24.0,ST,E. Haaland,96,80.0
5,24.0,ST,R. Lewandowski,93,87.0
7,24.0,ST,H. Kane,95,83.0
8,24.0,LW,Vini Jr.,89,90.0
12,24.0,LW,Neymar Jr,83,93.0
...,...,...,...,...,...
179938,15.0,ST,C. O'Connor,51,44.0
179949,15.0,ST,M. Bloomfield,48,50.0
179967,15.0,ST,S. Austin,48,48.0
179996,15.0,ST,A. Peterson,29,53.0


#### 1.3 Exploração Inicial.

Iremos calcular medidas de centralidade e dispersão categoria por categoria, analisando por ano e pegando a posição do jogador e sua categoria mais importante.

##### 1.3.1.Categoria Defensiva

In [41]:
for i in range(14, 25):
    # Filtrar jogadores que são defensores laterais (LB, RB, LWB, RWB) e têm a versão FIFA igual a i
    defense_fullback_year = defense_players[(defense_players['fifa_version'] == i) & 
                                             (defense_players['player_positions'].isin(["LB", "RB", "LWB", "RWB"]))][['movement_sprint_speed']]
    
    print(defense_fullback_year)



Empty DataFrame
Columns: [movement_sprint_speed]
Index: []
        movement_sprint_speed
163926                     87
163949                     83
163976                     74
163983                     79
163985                     85
...                       ...
179975                     62
179983                     60
179984                     63
179987                     73
179998                     69

[1195 rows x 1 columns]
        movement_sprint_speed
147188                     92
147208                     88
147220                     76
147224                     83
147242                     75
...                       ...
163810                     54
163813                     50
163819                     67
163824                     69
163827                     66

[1353 rows x 1 columns]
        movement_sprint_speed
129585                     93
129595                     80
129608                     89
129610                     83
129677               

In [39]:
for i in range(14,25):
    defense_fullback_year = defense_players[(defense_players['fifa_version'] == i) & 
                                            (defense_players['player_positions'].isin(["LB", "RB", "LWB", "RWB"])) & (defense_players['movement_sprint_speed'] > 0)]
    defense_fullback_year = defense_fullback_year[['fifa_version', 'player_positions', 'short_name', 'movement_sprint_speed']]
    defense_fullback_year = defense_fullback_year.describe()
    print(defense_fullback_year)
    
    



       fifa_version  movement_sprint_speed
count           0.0                    0.0
mean            NaN                    NaN
std             NaN                    NaN
min             NaN                    NaN
25%             NaN                    NaN
50%             NaN                    NaN
75%             NaN                    NaN
max             NaN                    NaN
       fifa_version  movement_sprint_speed
count        1195.0            1195.000000
mean           15.0              71.450209
std             0.0               9.382250
min            15.0              29.000000
25%            15.0              66.000000
50%            15.0              72.000000
75%            15.0              78.000000
max            15.0              93.000000
       fifa_version  movement_sprint_speed
count        1353.0            1353.000000
mean           16.0              72.070953
std             0.0               8.621186
min            16.0              30.000000
25%        